In [ ]:
from quant_utils.db_conn import DB_CONN_JJTG_DATA
import polars as pl
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

def query_fund_alpha(trade_dt: str) -> pl.DataFrame:
    query_sql = f"""
    SELECT
        END_DATE,
        TICKER_SYMBOL,
        `LEVEL`,
        INDICATOR,
        3M,
        6M,
        1Y,
        2Y,
        3Y
    FROM
        fund_derivatives_fund_alpha_performance 
    WHERE
        1 = 1 
        AND END_DATE = '{trade_dt}' 
    """
    df = DB_CONN_JJTG_DATA.exec_query(query_sql)
    df = pl.from_pandas(df)  # Convert Pandas DataFrame to Polars DataFrame
    df = df.unpivot(
        index=["END_DATE", "TICKER_SYMBOL", "LEVEL", "INDICATOR"],
        on=["3M", "6M", "1Y", "2Y", "3Y"],
        value_name="VALUE",
        variable_name="PERIOD"
    )
    return df

def query_fund_type(trade_dt: str) -> pl.DataFrame:
    query_sql = f"""
    SELECT
        TICKER_SYMBOL,
        LEVEL_1,
        LEVEL_2,
        LEVEL_3
    FROM
        fund_type_own_temp 
    WHERE
        1 = 1 
        AND REPORT_DATE = (
        SELECT
            max( REPORT_DATE ) 
        FROM
            fund_type_own_temp 
        WHERE
            PUBLISH_DATE <= '{trade_dt}'
    )
    """
    df = DB_CONN_JJTG_DATA.exec_query(query_sql)
    df = pl.from_pandas(df)  # Convert Pandas DataFrame to Polars DataFrame
    
    # 检查列是否存在
    if "TICKER_SYMBOL" not in df.columns or "LEVEL_1" not in df.columns or "LEVEL_2" not in df.columns or "LEVEL_3" not in df.columns:
        raise ValueError("Required columns not found in the DataFrame.")
    
    df = df.unpivot(
        index=["TICKER_SYMBOL"],
        on=["LEVEL_1", "LEVEL_2", "LEVEL_3"],
        value_name="LEVEL_NAME",
        variable_name="LEVEL"
    )
    return df

def query_fund_alpha_rank(trade_dt: str) -> pl.DataFrame:
    alpha_perf = query_fund_type(trade_dt).join(
        query_fund_alpha(trade_dt), 
        on=["TICKER_SYMBOL", "LEVEL"]
    )
    
    # Calculate rank
    alpha_perf = alpha_perf.with_columns(
        pl.col("VALUE")
        .over(["LEVEL", "LEVEL_NAME", "INDICATOR", "PERIOD"])
        .rank(method="min", descending=True)
        .alias("VALUE_RANK")
    )
    
    # Calculate percentage rank
    alpha_perf = alpha_perf.with_columns(
        (
            (pl.col("VALUE_RANK")-1) / (pl.col("VALUE_RANK").max()-1)
            .over(["LEVEL", "LEVEL_NAME", "INDICATOR", "PERIOD"])
        )
        .alias("VALUE")
    )
    
    # Adjust VALUE for specific indicators
    condition = pl.col("INDICATOR").is_in(["MAXDD", "ANNUAL_VOL"])
    alpha_perf = alpha_perf.with_columns(
        pl.when(condition)
        .then(1 - pl.col("VALUE"))
        .otherwise(pl.col("VALUE"))
        .alias("VALUE")
    )
    
    # Fill null values
    alpha_perf = alpha_perf.with_columns(
        pl.col("VALUE").fill_null(0.5)
    )
    
    return alpha_perf

def query_fund_alpha_score(trade_dt: str) -> pl.DataFrame:
    alpha_perf_rank = query_fund_alpha_rank(trade_dt)
    indicator_weight = {
        "CUM_ALPHA": 0.35,
        "IR": 0.35,
        "MAXDD": 0.3,
        "ANNUAL_VOL": 0.0,
    }
    period_weight = {
        "3M": 0.2,
        "6M": 0.2,
        '1Y': 0.2,
        '2Y': 0.2,
        '3Y': 0.2,
    }
    
    # Initialize INDICATOR_WEIGHT and PERIOD_WEIGHT columns
    alpha_perf_rank = alpha_perf_rank.with_columns(
        pl.lit(0.0).alias("INDICATOR_WEIGHT"),
        pl.lit(0.0).alias("PERIOD_WEIGHT")
    )
    
    # Apply indicator weights
    for indicator, weight in indicator_weight.items():
        alpha_perf_rank = alpha_perf_rank.with_columns(
            pl.when(pl.col("INDICATOR") == indicator)
            .then(weight)
            .otherwise(pl.col("INDICATOR_WEIGHT"))
            .alias("INDICATOR_WEIGHT")
        )
    
    # Apply period weights
    for period, weight in period_weight.items():
        alpha_perf_rank = alpha_perf_rank.with_columns(
            pl.when(pl.col("PERIOD") == period)
            .then(weight)
            .otherwise(pl.col("PERIOD_WEIGHT"))
            .alias("PERIOD_WEIGHT")
        )
    
    # Calculate weighted value
    alpha_perf_rank = alpha_perf_rank.with_columns(
        pl.col("VALUE") * pl.col("INDICATOR_WEIGHT") * pl.col("PERIOD_WEIGHT")
    )
    # Aggregate by TICKER_SYMBOL, LEVEL, LEVEL_NAME
    alpha_perf_score = (
        alpha_perf_rank
        .group_by(["TICKER_SYMBOL", "LEVEL", "LEVEL_NAME"])
        .agg(pl.col("VALUE").sum().alias("VALUE"))
        .with_columns(pl.col("VALUE").fill_null(np.nan))
    )
    
    return alpha_perf_score

def query_fund_alpha_score_rank(trade_dt: str) -> pl.DataFrame:
    alpha_perf_score = query_fund_alpha_score(trade_dt)
    
    # Calculate rank
    alpha_perf_score = alpha_perf_score.with_columns(
        pl.col("VALUE")
        .over(["LEVEL", "LEVEL_NAME"])
        .rank(method="min", descending=True)
        .alias("RANK")
    )
    
    # Calculate percentage rank
    alpha_perf_score = alpha_perf_score.with_columns(
        (pl.col("RANK") / pl.col("RANK").max().over(["LEVEL", "LEVEL_NAME"]))
        .alias("RANK")
    )
    
    # Multiply rank by 100
    alpha_perf_score = alpha_perf_score.with_columns(
        pl.col("RANK") * 100
    )
    
    return alpha_perf_score

df = query_fund_alpha_score_rank("20241114")

In [1]:
import os
import pandas as pd
from dateutil.parser import parse
import polars as pl
import duckdb
import numpy as np
from quant_utils.constant import DB_CONFIG
ftr_path = "F:/data_ftr/fund_nav/"
ftr_list = os.listdir(ftr_path)
parquet_path = "F:/data_parquet/fund_nav/"
DB_CONFIG["jjtg"]
def crate_database_uri(config: dict) -> str:
    return f"mysql://{config['user']}:{config['pwd']}@{config['host']}:{config['port']}/{config['database']}"
jjtg_uri = crate_database_uri(DB_CONFIG["jjtg"])

In [2]:
def get_fund_nav_by_parquet(
        start_date: str, 
        end_date: str,
        parquet_path: str = "F:/data_parquet/fund_nav/"
) -> pl.DataFrame:
    start_date = parse(start_date).strftime("%Y-%m-%d")
    end_date = parse(end_date).strftime("%Y-%m-%d")
    query = f"""
        SELECT 
            END_DATE, 
            TICKER_SYMBOL, 
            ADJ_NAV
        FROM 
            '{parquet_path}*.parquet' 
        where 
            1=1
            and END_DATE between '{start_date}' and '{end_date}'
        order by
            END_DATE,
            TICKER_SYMBOL
    """
    with duckdb.connect() as con:
        df = con.sql(query).pl()
    return df

def get_trade_cal_by_parquet(
        parquet_path: str = "F:/data_parquet/"
) -> pl.DataFrame:
    query = f"""
        SELECT
            *
        FROM
            '{parquet_path}trade_cal.parquet'
    """
    with duckdb.connect() as con:
        df = con.sql(query).pl()
    return df

In [ ]:
import quant_utils.data_moudle as dm


In [ ]:
def unpivot_dataframe(df: pl.LazyFrame) -> pl.LazyFrame:
    """
    数据透视表转置
    """
    return df.unpivot(
        index=["TICKER_SYMBOL", "START_DATE", "END_DATE"],
        variable_name="INDICATOR",
        value_name="PORTFOLIO_VALUE",
        on=[
            'CUM_RETURN',
            'ANNUAL_RETURN',
            'ANNUAL_VOLATILITY',
            'SHARP_RATIO_ANNUAL',
            'CALMAR_RATIO_ANNUAL',
            'MAXDD' 
        ]
    )
def get_portfolio_performance(
        portfolio_name: str,
        end_date: str, 
        table_name:str,
    ) -> pl.LazyFrame:
    query_sql = f"""
    SELECT
        a.TICKER_SYMBOL,
        a.START_DATE,
        a.END_DATE,
        a.CUM_RETURN,
        a.ANNUAL_RETURN,
        a.ANNUAL_VOLATILITY,
        a.SHARP_RATIO_ANNUAL,
        a.CALMAR_RATIO_ANNUAL,
        a.MAXDD
    FROM
        {table_name} a
    WHERE
        1 = 1
        AND a.END_DATE = '{end_date}'
        AND a.TICKER_SYMBOL = '{portfolio_name}'
    """
    return pl.read_database_uri(query_sql, uri=jjtg_uri).lazy().pipe(unpivot_dataframe)

def get_peer_fund_performance(
        portfolio_name: str,
        end_date: str, 
    ) -> pl.LazyFrame:
    peer_query = (
        dm.get_portfolio_info()
        .query(f"PORTFOLIO_NAME == '{portfolio_name}'")["PEER_QUERY"]
        .values[0]
    )
    peer_query = peer_query.replace("LEVEL", "c.LEVEL")
    peer_query = peer_query.replace("==", "=")
    query_sql = f"""
    SELECT
        a.TICKER_SYMBOL,
        a.START_DATE,
        a.END_DATE,
        a.CUM_RETURN,
        a.ANNUAL_RETURN,
        a.ANNUAL_VOLATILITY,
        a.SHARP_RATIO_ANNUAL,
        a.CALMAR_RATIO_ANNUAL,
        a.MAXDD 
    FROM
        fund_performance_inner a
        JOIN fund_type_own c ON c.TICKER_SYMBOL = a.TICKER_SYMBOL
    WHERE
        1 = 1 
        and {peer_query}
        AND a.END_DATE = '{end_date}' 
        AND ( 
            c.REPORT_DATE = ( 
                SELECT max( report_date ) 
                FROM fund_type_own 
                WHERE PUBLISH_DATE <= '{end_date}' 
            )
        )
    """
    return pl.read_database_uri(query_sql, uri=jjtg_uri).lazy().pipe(unpivot_dataframe)

def get_peer_fof_performance(
        portfolio_name: str,
        end_date: str,
    ) -> pl.LazyFrame:
    query_sql = f"""
    SELECT
        a.TICKER_SYMBOL,
        a.START_DATE,
        a.END_DATE,
        a.CUM_RETURN,
        a.ANNUAL_RETURN,
        a.ANNUAL_VOLATILITY,
        a.SHARP_RATIO_ANNUAL,
        a.CALMAR_RATIO_ANNUAL,
        a.MAXDD 
    FROM
        fund_performance_inner a
        JOIN fof_type c ON c.TICKER_SYMBOL = a.TICKER_SYMBOL 
    WHERE
        1 = 1 
        AND a.END_DATE = '{end_date}' 
        AND c.INNER_TYPE = '{portfolio_name}' 
    """
    return pl.read_database_uri(query_sql, uri=jjtg_uri).lazy().pipe(unpivot_dataframe)

def get_peer_portfolio_performance(
        portfolio_name: str,
        end_date: str,
):
    query_sql = f"""
    SELECT
        a.TICKER_SYMBOL,
        a.START_DATE,
        a.END_DATE,
        a.CUM_RETURN,
        a.ANNUAL_RETURN,
        a.ANNUAL_VOLATILITY,
        a.SHARP_RATIO_ANNUAL,
        a.CALMAR_RATIO_ANNUAL,
        a.MAXDD 
    FROM
        peer_performance_inner a
        JOIN peer_portfolio_type c ON c.TICKER_SYMBOL = a.TICKER_SYMBOL 
    WHERE
        1 = 1 
        AND a.END_DATE = '{end_date}' 
        AND c.PORTFOLIO_TYPE = '{portfolio_name}'
    """
    return pl.read_database_uri(query_sql, uri=jjtg_uri).lazy().pipe(unpivot_dataframe)

def get_benchmark_value_outter(portfolio_name: str, end_date: str) -> pl.lazyframe:
    query = f"""
    SELECT
        a.TICKER_SYMBOL,
        a.START_DATE,
        a.END_DATE,
        a.CUM_RETURN,
        a.ANNUAL_RETURN,
        a.ANNUAL_VOLATILITY,
        a.SHARP_RATIO_ANNUAL,
        a.CALMAR_RATIO_ANNUAL,
        a.MAXDD 
    FROM
        benchmark_performance_inner a
    WHERE
        1 = 1 
        AND a.END_DATE = '{end_date}' 
        AND a.TICKER_SYMBOL = '{portfolio_name}' 
    """
    df = pl.read_database_uri(query, uri=jjtg_uri).lazy()
    df_unpivot =  df.unpivot(
        index=["TICKER_SYMBOL", "START_DATE","END_DATE"],
        variable_name="INDICATOR",
        value_name="BENCHMARK_VALUE_OUTER",
    )
    return df_unpivot
def rank_pct(
        rank_col: str, 
        patition_by: str|list = None, 
        descending: bool = True
) -> pl.Expr:
    rank_expr = pl.col(rank_col).rank(descending=descending).cast(pl.UInt32)
    count_expr = pl.col(rank_col).count().cast(pl.UInt32)
    return 100*((rank_expr-1)/(count_expr-1)).over(patition_by)

def rank_str(
        rank_col: str,
        patition_by: str|list = None,
        descending: bool = True
) -> pl.Expr:
    rank_expr = pl.col(rank_col).rank(descending=descending).cast(pl.UInt32)
    count_expr = pl.col(rank_col).count().cast(pl.UInt32)
    return rank_expr.cast(pl.String).over(patition_by) + '/' + count_expr.cast(pl.String).over(patition_by)

def _cal_performance_rank_helper(
        df_unpivot: pl.LazyFrame,
        patition_by: str|list = None,
        incicator_list: list = None,
        descending: bool = True,
    ) -> pl.LazyFrame:
    # 计算排名及百分位
    # 特别注意在polars中rank函数不考虑空值
    result_df = (
        df_unpivot
        .select(
            [
                pl.col("TICKER_SYMBOL"),
                pl.col("START_DATE"),
                pl.col("END_DATE"),
                pl.col('INDICATOR'),
                pl.col("PORTFOLIO_VALUE"),
            ]
        )
        .filter(pl.col("INDICATOR").is_in(incicator_list))
        .with_columns(
            rank_pct("PORTFOLIO_VALUE", patition_by=patition_by, descending=descending).alias("PEER_RANK_PCT"),
            rank_str("PORTFOLIO_VALUE", patition_by=patition_by, descending=descending).alias("PEER_RANK"),
        )
    )

    return result_df
def cal_performance_rank(df:pl.LazyFrame, portfolio_name:str)->pl.LazyFrame:
    # df_unpivot = df.unpivot(
    #     index=["TICKER_SYMBOL", "START_DATE", "END_DATE"],
    #     variable_name="INDICATOR",
    #     value_name="PORTFOLIO_VALUE",
    # )
    asscending_indicators = ["MAXDD", "ANNUAL_VOLATILITY"]
    descending_indicators = ["CUM_RETURN", "ANNUAL_RETURN", "SHARP_RATIO_ANNUAL", "CALMAR_RATIO_ANNUAL"]
    patition_by=["START_DATE", "END_DATE", "INDICATOR"]
    # 计算排名及百分位
    # 特别注意在polars中rank函数不考虑空值
    df_asscending = _cal_performance_rank_helper(
        df,
        patition_by=patition_by,
        incicator_list=asscending_indicators,
        descending=False,
    )

    df_descending = _cal_performance_rank_helper(
        df,
        patition_by=patition_by,
        incicator_list=descending_indicators,
        descending=True,
    )
    result = pl.concat([df_asscending, df_descending]).filter(pl.col("TICKER_SYMBOL") == portfolio_name)
    return result

def get_portfolio_dates(portfolio_name:str, end_date:str)->pl.LazyFrame:
    query_sql = f"""
    SELECT
        DATE_NAME AS CYCLE,
        START_DATE,
        END_DATE 
    FROM
        portfolio_dates 
    WHERE
        1 = 1 
        AND (PORTFOLIO_NAME = '{portfolio_name}' OR PORTFOLIO_NAME = 'ALL') 
        AND END_DATE = '{end_date}'
    """
    return pl.read_database_uri(query_sql, uri=jjtg_uri).lazy()
def rename_indicator_col_into_chinese(
    df: pl.LazyFrame
):
    indicator_map_dict = {
        "CUM_RETURN": "累计收益率",
        "ANNUAL_RETURN": "年化收益率",
        "ANNUAL_VOLATILITY": "年化波动率",
        "SHARP_RATIO_ANNUAL": "收益波动比",
        "CALMAR_RATIO_ANNUAL": "年化收益回撤比",
        "MAXDD": "最大回撤"
    }
    return df.with_columns(
        pl.col("INDICATOR").replace(
            indicator_map_dict
        ).alias("INDICATOR")
    )

def add_benchmark_value_otter(
        df:pl.LazyFrame, portfolio_name:str, end_date:str
    )->pl.LazyFrame:
    benchmark_df = get_benchmark_value_outter(portfolio_name, end_date)
    # print("benchmark_df", benchmark_df.collect())
    result = df.join(
        benchmark_df, 
        on=["TICKER_SYMBOL","START_DATE", "END_DATE", "INDICATOR"],
        how="left"
    )
    # print("result", result.collect())
    return result

def add_peer_fof_performance(
    df:pl.LazyFrame, portfolio_name:str, end_date:str
    )->pl.LazyFrame:
    peer_fof = get_peer_fof_performance(
        portfolio_name, end_date
    )
    result = cal_performance_rank(pl.concat([df, peer_fof]), portfolio_name)
    result = result.rename(
        {
            "PEER_RANK_PCT": "PEER_FOF_RANK_PCT",
            "PEER_RANK": "PEER_FOF_RANK"
        }
    )
    result = result.select(
        [
            "TICKER_SYMBOL",
            "START_DATE",
            "END_DATE",
            "INDICATOR",
            "PEER_FOF_RANK_PCT",
            "PEER_FOF_RANK"
        ]
    )
    return df.join(
        result,
        on=["TICKER_SYMBOL","START_DATE", "END_DATE", "INDICATOR"],
        how="left"
    )
def add_peer_portfolio_performance(
    df:pl.LazyFrame, portfolio_name:str, end_date:str
)->pl.LazyFrame:
    peer_portfolio = get_peer_portfolio_performance(
        portfolio_name, end_date
    )
    result = cal_performance_rank(pl.concat([df, peer_portfolio]), portfolio_name)
    result = result.rename(
        {
            "PEER_RANK_PCT": "PEER_FOF_PORTFOLIO_PCT",
            "PEER_RANK": "PEER_PORTFOLIO_RANK"
        }
    )
    result = result.select(
        [
            "TICKER_SYMBOL",
            "START_DATE",
            "END_DATE",
            "INDICATOR",
            "PEER_FOF_PORTFOLIO_PCT",
            "PEER_PORTFOLIO_RANK"
        ]
    )
    return df.join(
        result,
        on=["TICKER_SYMBOL","START_DATE", "END_DATE", "INDICATOR"],
        how="left"
    )

def cal_peer_median(peer_fund_performance:pl.LazyFrame)->pl.LazyFrame:

    peer_median = (
        peer_fund_performance
        .group_by(
            ["START_DATE", "END_DATE", "INDICATOR"]
        )
        .agg(
            pl.col("PORTFOLIO_VALUE").median().alias("PEER_MEDIAN")
        )
    )
    return peer_median
def _cal_portfolio_performance(
    portfolio_name:str, end_date:str, table_name:str
)->pl.LazyFrame:


    portfolio_perf = get_portfolio_performance(
        portfolio_name, 
        end_date,
        table_name
    )
  
    # print("portfolio_perf", portfolio_perf.collect())
    peer_fund_performance = get_peer_fund_performance(
        portfolio_name, end_date
    )

    peer_median = cal_peer_median(peer_fund_performance)

    df = pl.concat(
        [portfolio_perf, peer_fund_performance]
    )
    perf_rank = cal_performance_rank(df, portfolio_name).join(
        peer_median, 
        on=["START_DATE", "END_DATE", "INDICATOR"],
        how="left"
    )
    # perf_rank = portfolio_dates.join(
    #     perf_rank,
    #     on=["START_DATE", "END_DATE"]
    # )
    # print("perf_rank", perf_rank.collect())
    return perf_rank

def get_portfolio_derivatives_rank(portfolio_name:str, end_date:str):

    portfolio_dates = get_portfolio_dates(
        portfolio_name, end_date
    )
    perf_rank = _cal_portfolio_performance(
        portfolio_name, end_date, "portfolio_derivatives_performance_inner"
    )
    perf_rank = (
        perf_rank
        .pipe(add_benchmark_value_otter, portfolio_name, end_date)
        .rename({"BENCHMARK_VALUE_OUTER": "BENCHMARK_VALUE_INNER"})
        .pipe(rename_indicator_col_into_chinese)
    )
    return portfolio_dates.join(
        perf_rank,
        on=["START_DATE", "END_DATE"]
    )

def get_portfolio_rank(portfolio_name:str, end_date:str):
    portfolio_dates = get_portfolio_dates(
        portfolio_name, end_date
    )
    perf_rank = _cal_portfolio_performance(
        portfolio_name, end_date, "portfolio_performance_inner"
    )
    perf_rank = (
        perf_rank
        .pipe(add_benchmark_value_otter, portfolio_name, end_date)
        .pipe(add_peer_fof_performance, portfolio_name, end_date)
        .pipe(add_peer_portfolio_performance, portfolio_name, end_date)
        .pipe(rename_indicator_col_into_chinese)
    )
    return portfolio_dates.join(
        perf_rank,
        on=["START_DATE", "END_DATE"]
    )

In [66]:
df = get_portfolio_derivatives_rank(
    "知己优选-进取全明星", "20241122"
).collect()

df1 = get_portfolio_rank(
    "知己优选-进取全明星", "20241122"
).collect()

In [67]:
df

CYCLE,START_DATE,END_DATE,TICKER_SYMBOL,INDICATOR,PORTFOLIO_VALUE,PEER_RANK_PCT,PEER_RANK,PEER_MEDIAN,BENCHMARK_VALUE_INNER
str,date,date,str,str,f64,f64,str,f64,f64
"""成立日""",2021-09-03,2024-11-22,"""知己优选-进取全明星""","""年化波动率""",16.512071,8.117673,"""368/4522""",21.743427,17.750713
"""近3年""",2021-11-22,2024-11-22,"""知己优选-进取全明星""","""年化波动率""",16.732227,8.335176,"""378/4524""",21.852659,18.028628
"""对客日""",2022-01-24,2024-11-22,"""知己优选-进取全明星""","""年化波动率""",16.913945,8.428011,"""409/4842""",22.095198,18.244836
"""近2年""",2022-11-22,2024-11-22,"""知己优选-进取全明星""","""年化波动率""",15.83698,9.929672,"""594/5973""",21.082166,17.216314
"""近1年""",2023-11-22,2024-11-22,"""知己优选-进取全明星""","""年化波动率""",19.492946,17.156371,"""1198/6978""",24.912297,21.247914
…,…,…,…,…,…,…,…,…,…
"""TGDS_2""",2024-09-30,2024-11-22,"""知己优选-进取全明星""","""年化收益回撤比""",-2.484421,50.570984,"""3898/7707""",-2.433275,-2.025208
"""近1月""",2024-10-22,2024-11-22,"""知己优选-进取全明星""","""年化收益回撤比""",-3.972893,46.030295,"""3526/7659""",-4.363493,-4.042733
"""MTD""",2024-10-31,2024-11-22,"""知己优选-进取全明星""","""年化收益回撤比""",-3.711827,50.143603,"""3842/7661""",-3.690076,-3.269077


In [68]:
df1

CYCLE,START_DATE,END_DATE,TICKER_SYMBOL,INDICATOR,PORTFOLIO_VALUE,PEER_RANK_PCT,PEER_RANK,PEER_MEDIAN,BENCHMARK_VALUE_OUTER,PEER_FOF_RANK_PCT,PEER_FOF_RANK,PEER_FOF_PORTFOLIO_PCT,PEER_PORTFOLIO_RANK
str,date,date,str,str,f64,f64,str,f64,f64,f64,str,f64,str
"""成立日""",2021-09-03,2024-11-22,"""知己优选-进取全明星""","""年化波动率""",16.511113,8.117673,"""368/4522""",21.743427,17.750713,NaN,"""1/1""",NaN,"""1/1"""
"""近3年""",2021-11-22,2024-11-22,"""知己优选-进取全明星""","""年化波动率""",16.731223,8.335176,"""378/4524""",21.852659,18.028628,NaN,"""1/1""",NaN,"""1/1"""
"""对客日""",2022-01-24,2024-11-22,"""知己优选-进取全明星""","""年化波动率""",16.912975,8.428011,"""409/4842""",22.095198,18.244836,0.0,"""1/5""",NaN,"""1/1"""
"""近2年""",2022-11-22,2024-11-22,"""知己优选-进取全明星""","""年化波动率""",15.835719,9.929672,"""594/5973""",21.082166,17.216314,0.0,"""1/7""",40.776699,"""85/207"""
"""近1年""",2023-11-22,2024-11-22,"""知己优选-进取全明星""","""年化波动率""",19.492657,17.156371,"""1198/6978""",24.912297,21.247914,0.0,"""1/7""",49.767442,"""108/216"""
…,…,…,…,…,…,…,…,…,…,…,…,…,…
"""TGDS_2""",2024-09-30,2024-11-22,"""知己优选-进取全明星""","""年化收益回撤比""",-2.484419,50.570984,"""3898/7707""",-2.433275,-2.025208,100.0,"""7/7""",48.372093,"""105/216"""
"""近1月""",2024-10-22,2024-11-22,"""知己优选-进取全明星""","""年化收益回撤比""",-3.972889,46.030295,"""3526/7659""",-4.363493,-4.042733,100.0,"""7/7""",43.255814,"""94/216"""
"""MTD""",2024-10-31,2024-11-22,"""知己优选-进取全明星""","""年化收益回撤比""",-3.711839,50.143603,"""3842/7661""",-3.690076,-3.269077,100.0,"""7/7""",58.604651,"""127/216"""
